In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
# To check whether the pipeline is running initial load or incremental load
dbutils.widgets.text("incremental_flag", "0")
dbutils.widgets.get("incremental_flag")

In [0]:
incremental_flag = dbutils.widgets.get("incremental_flag")
incremental_flag

In [0]:
# Connecting to Azure Data Lake 
spark.conf.set('fs.azure.account.key.datalakeazuredatabricks.dfs.core.windows.net', 'LuUWQy2pFP32+FiO94ok2cwkoyLWOI2zO2kFBtsCHBqt4t/ohW8squAMNX7wPMEzMebxoaHk3k5o+AStROC22A==')

In [0]:
%sql
select * from parquet.`abfss://silver@datalakeazuredatabricks.dfs.core.windows.net`

In [0]:
filename = dbutils.fs.ls('abfss://silver@datalakeazuredatabricks.dfs.core.windows.net')[3].name
filename

In [0]:
%sql
-- Customer Dimension
select distinct(CustomerID) as CustomerID, CustomerName, CustomerEmail, Country
from parquet.`abfss://silver@datalakeazuredatabricks.dfs.core.windows.net`

In [0]:
# Converting sql query into dataframe
df_src = spark.sql(
    '''select distinct(CustomerID) as CustomerID, CustomerName, CustomerEmail, Country
from parquet.`abfss://silver@datalakeazuredatabricks.dfs.core.windows.net`'''
)
df_src.display()

In [0]:
# for gold layer data, creating a new dataframe with only required columns for gold.customerdimension & verify with source for incremental load
path = 'abfss://gold@datalakeazuredatabricks.dfs.core.windows.net'

if DeltaTable.isDeltaTable(spark, path):
    df_sink = spark.sql(
        '''select Dim_Customer_Key, CustomerID, CustomerName, CustomerEmail, Country from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net`'''
    )
    print('Table exists')
else:
    df_sink = spark.createDataFrame([], schema='Dim_Customer_Key int, CustomerID int, CustomerName string, CustomerEmail string, Country string')
    # df_sink = spark.sql(
    #     '''
    #     select 1 as Dim_Customer_Key, CustomerID, CustomerName, CustomerEmail, Country from parquet.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net` where 1 = 2
    #     '''
    # )
# df_sink = spark.createDataFrame([], schema='Dim_Customer_Key int, CustomerID int, CustomerName string, CustomerEmail string, Country string')
df_sink.display()


In [0]:
df = df_src.join(df_sink, df_src['CustomerID'] == df_sink['CustomerID'], 'left').select(df_src['CustomerID'], df_src['CustomerName'], df_src['CustomerEmail'], df_src['Country'], df_sink['Dim_Customer_Key'])

df.display()
# df = df_src.join(df_sink,)

In [0]:
# Old Records
df_old = df.filter(df['Dim_Customer_Key'].isNotNull())
df_old.display()


In [0]:
# New Records
df_new = df.filter(df['Dim_Customer_Key'].isNull())
df_new.display()


In [0]:
# Get maximum Dim_Customer_Key
if incremental_flag == '0':
    max_value = 1
else:
    max_value = df_old.agg(max(df_old['Dim_Customer_Key'])).collect()[0][0]
max_value 

In [0]:
# Creating surrogate key for new records
df = df.withColumn('Dim_Customer_Key', max_value + monotonically_increasing_id())
df.display()

In [0]:
# Union old_data and new_data
df = df_old.union(df)
df.display()

In [0]:
# SCD Type1 = Upsert or Update + Insert
from delta.tables import DeltaTable

table_name = 'gold.dim_Customer'
path = 'abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Customer'

if DeltaTable.isDeltaTable(spark, path):
    deltaTable = DeltaTable.forPath(spark, 'abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Customer')
    deltaTable.alias('target').merge(df.alias('source'), 'target.Dim_Customer_Key = source.Dim_Customer_Key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df.write.format('delta').mode('overwrite').option('mergeSchema', 'true').save(path)
    print('Table is created')


In [0]:
%sql
select * from delta.`abfss://gold@datalakeazuredatabricks.dfs.core.windows.net/dim_Customer`